In [ ]:


# Install necessary package
!pip install python-docx

# Import Libraries
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from docx import Document
from docx.shared import Inches

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths and parameters
input_path = 'path'
output_path = 'path'
os.makedirs(output_path, exist_ok=True)

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.0001

# Create Word document
doc = Document()
doc.add_heading('DenseNet169 Classification Report', 0)

# Model creation function
def create_densenet169(input_shape, num_classes):
    base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Data loader
def process_data(input_path, img_size, batch_size):
    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.densenet.preprocess_input,
        validation_split=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest"
    )
    train_gen = datagen.flow_from_directory(
        input_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    val_gen = datagen.flow_from_directory(
        input_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    return train_gen, val_gen

# Start training DenseNet169
model_name = 'DenseNet169'
img_size = (224, 224)
train_gen, val_gen = process_data(input_path, img_size, BATCH_SIZE)
model = create_densenet169((img_size[0], img_size[1], 3), train_gen.num_classes)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint_path = os.path.join(output_path, f'{model_name}_best_model.keras')
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

history = model.fit(train_gen, epochs=EPOCHS, validation_data=val_gen, callbacks=[checkpoint])

# Plot accuracy & loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title(f"{model_name} Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title(f"{model_name} Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
acc_loss_path = os.path.join(output_path, f'{model_name}_accuracy_loss_curve.png')
plt.savefig(acc_loss_path)
plt.close()

# Evaluation
model.load_weights(checkpoint_path)
eval_result = model.evaluate(val_gen)
Y_pred = model.predict(val_gen)
y_pred = np.argmax(Y_pred, axis=1)
y_true = val_gen.classes
report = classification_report(y_true, y_pred, target_names=list(train_gen.class_indices.keys()))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
cm_path = os.path.join(output_path, f'{model_name}_confusion_matrix.png')
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=train_gen.class_indices.keys(),
            yticklabels=train_gen.class_indices.keys(),
            annot_kws={"size": 14})
plt.xticks(rotation=45, fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.title(f'{model_name} Confusion Matrix', fontsize=16)
plt.tight_layout()
plt.savefig(cm_path)
plt.close()

# Add results to Word doc
doc.add_heading(f'{model_name} Performance', level=1)
doc.add_paragraph(f"Final Validation Loss: {eval_result[0]:.4f}")
doc.add_paragraph(f"Final Validation Accuracy: {eval_result[1]:.4f}")
doc.add_heading(f'{model_name} Classification Metrics', level=2)
doc.add_paragraph(report)
doc.add_heading(f'{model_name} Training History', level=2)
doc.add_picture(acc_loss_path, width=Inches(6))
doc.add_heading(f'{model_name} Confusion Matrix', level=2)
doc.add_picture(cm_path, width=Inches(6))
doc.save(os.path.join(output_path, f'{model_name}_classification_report.docx'))

print("DenseNet169 training and evaluation complete. Results saved.")
